In [1]:
from functools import wraps

In [2]:
# Decorator to prime coroutine
def coroutine(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        cr = func(*args, **kwargs)
        next(cr) # No need to prime the coroutine anymore, the decorator does it for you
        return cr
    return wrapper

In [3]:
def example_grep(pattern):
    print('Looking for {}'.format(pattern))
    while True:
        line = yield
        if pattern in line:
            print(line)

In [4]:
g = example_grep('python')
next(g) # Or just use the decorator above for auto priming

Looking for python


In [5]:
g.send('Hello World')
g.send('I am a pythonista.')
g.send('c++ or python')
g.send('never mind')

I am a pythonista.
c++ or python


In [6]:
g.close() # Stop the generator. Will throw an exception that you need to catch

In [7]:
@coroutine
def printer():
    while True:
        line = yield
        print(line)

In [8]:
@coroutine
def broadcast(targets):
    while True:
        line = yield
        for target in targets:
            target.send(line)

In [9]:
@coroutine
def grep(pattern, publisher):
    while True:
        line = yield
        if pattern in line:
            publisher.send(line+' ({})'.format(pattern))

In [10]:
p = printer()
x = grep('python', p)
y = grep('ml', p)
b = broadcast([x, y])

In [11]:
b.send('Hello')
b.send('All hail the BDFL')
b.send('python is cool')
b.send('ml is in demand')
b.send('C++ or Fortran')
b.send('ml uses python') # This will be repeated

python is cool (python)
ml is in demand (ml)
ml uses python (python)
ml uses python (ml)
